In [ ]:
import napari
import sys
import numpy as np
# Databases saved with numpy>2 need np._core.numeric, which is not available in numpy<2, hence the following hack
sys.modules["numpy._core.numeric"] = np.core.numeric
from pathlib import Path
from trackedit.motile_overwrites import *
from trackedit.TrackEditClass import *
from trackedit.DatabaseHandler import DatabaseHandler
from motile_tracker.data_model.actions import AddEdges, DeleteNodes, DeleteEdges, AddNodes


#**********INPUTS*********
working_directory = Path('/home/teun.huijben/Documents/data/Akila/20241003/neuromast4_t851/adjusted/')
db_filename_old = 'data.db'
data_shape_full = [851,73,1024,1024]      #T,(Z),Y,X       (851,73,1024,1024)
scale = (4,1,1)
layer_name = 'ultrack'
allow_overwrite = False      #overwrite existing database/changelog
#*************************

DB_handler = DatabaseHandler(
            db_filename_old = db_filename_old,
            working_directory = working_directory,
            data_shape_full = data_shape_full,
            scale = scale[0],
            name = 'ultrack',
            allow_overwrite = allow_overwrite)

#overwrite some motile functions
DeleteNodes._apply = create_db_delete_nodes(DB_handler)
DeleteEdges._apply = create_db_delete_edges(DB_handler)
AddEdges._apply = create_db_add_edges(DB_handler)
AddNodes._apply = create_db_add_nodes(DB_handler)
TracksViewer._refresh = create_tracks_viewer_and_segments_refresh(layer_name=layer_name)


#open napari with TrackEdit
viewer = napari.Viewer()
trackeditclass = TrackEditClass(viewer, databasehandler = DB_handler)
viewer.dims.ndisplay = 3    #3D view